In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import torch
torch.manual_seed(42)  # Set to any fixed integer


torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)  # If using multiple GPUs
import numpy as np
np.random.seed(42)


In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'
import numpy as np
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


import matplotlib.pyplot as plt

cuda:0


In [4]:
T_matrix=torch.tensor([[0.9, 0.1, 0.0,0.0],
        [0.00, 0.9, 0.1,0],
        [0.0, 0.00, 0.9,.1],
                      [0.0,0.0,0.0,1]], device=device) 

In [5]:
T_matrix1=torch.zeros(4,4,4)
T_matrix1[0,:,:]=T_matrix+torch.eye(4,4)*-0.0
T_matrix1[1,:,:]=T_matrix+torch.eye(4,4)*-0.3
T_matrix1[2,:,:]=T_matrix+torch.eye(4,4)*-0.5
T_matrix1[3,:,:]=T_matrix+torch.eye(4,4)*-0.7

In [6]:
T_matrix1=T_matrix1/torch.sum(T_matrix1, 2).reshape(4,4,1)

In [7]:
T_matrix1

tensor([[[0.9000, 0.1000, 0.0000, 0.0000],
         [0.0000, 0.9000, 0.1000, 0.0000],
         [0.0000, 0.0000, 0.9000, 0.1000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.8571, 0.1429, 0.0000, 0.0000],
         [0.0000, 0.8571, 0.1429, 0.0000],
         [0.0000, 0.0000, 0.8571, 0.1429],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.8000, 0.2000, 0.0000, 0.0000],
         [0.0000, 0.8000, 0.2000, 0.0000],
         [0.0000, 0.0000, 0.8000, 0.2000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.6667, 0.3333, 0.0000, 0.0000],
         [0.0000, 0.6667, 0.3333, 0.0000],
         [0.0000, 0.0000, 0.6667, 0.3333],
         [0.0000, 0.0000, 0.0000, 1.0000]]])

In [8]:

s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)



print(T_matrix)

print(s_0)

tensor([[0.9000, 0.1000, 0.0000, 0.0000],
        [0.0000, 0.9000, 0.1000, 0.0000],
        [0.0000, 0.0000, 0.9000, 0.1000],
        [0.0000, 0.0000, 0.0000, 1.0000]])
tensor([1., 0., 0.])


In [9]:
import numpy as np

# Define the joint state space for two units (4 states each)
joint_states = [(i, j) for i in range(4) for j in range(4)]
num_joint_states = len(joint_states)
# Sort the joint states based on the sum of values in each tuple
joint_states = sorted(joint_states, key=lambda x: sum(x))
# Example: Define a 4x4 joint transition matrix P((s1, s2) -> (s1', s2'))
P_joint = np.array([
    # Each row corresponds to a joint state, each column to the next joint state
    [0.4, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],  # (0,0) -> various states
    [0.3, 0.3, 0.1, 0.1, 0.07, 0.05, 0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (0,1)
    [0.2, 0.2, 0.3, 0.1, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.03, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,2)
    [0.1, 0.1, 0.2, 0.4, 0.05, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,3)
    [0.3, 0.2, 0.1, 0.1, 0.2, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (1,0)
    [0.2, 0.2, 0.1, 0.1, 0.3, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0],  # (1,1)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.4, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.01, 0.0, 0.0],  # (1,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.2, 0.1, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0],  # (1,3)
    [0.3, 0.1, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01],  # (2,0)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.1, 0.4, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01],  # (2,1)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.05, 0.05, 0.3, 0.05, 0.02, 0.02, 0.01, 0.01],  # (2,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.1, 0.3, 0.05, 0.05, 0.02, 0.02],  # (2,3)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.3, 0.1, 0.05, 0.05],  # (3,0)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.3, 0.05, 0.1],  # (3,1)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.05, 0.1, 0.4, 0.2],  # (3,2)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.1, 0.2, 0.3],  # (3,3)
])

# Set lower triangular elements to 0
for i in range(16):
    for j in range(i):
        P_joint[i, j] = 0
# Set lower triangular elements to 0
for i in range(16):
    for j in range(i+5,16):
        P_joint[i, j] = 0

P_joint=P_joint+np.eye(16,16)*0.5
# Normalize each row so the sum of probabilities equals 1
P_joint = P_joint / P_joint.sum(axis=1, keepdims=True)

# Check if the rows sum to 1
row_sums = P_joint.sum(axis=1)
print(row_sums)  # Should all be close to 1


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [10]:

# import torch
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(42)



In [11]:
from hmmlearn import hmm
import numpy as np
import torch

# Load data
# Load data
pred1 = torch.load('TRAIN_DATA_NON_LINEAR.pth')

# Extract observations and underlying states
observations_o = [o.numpy().reshape(-1, 1) for o, _, _ in pred1]
observations_o1 = [o.numpy().reshape(-1, 1) for _, o, _ in pred1]
true_states = 0
true_states1 = 0

import numpy as np
import ssm

def train_hmm_and_evaluate(observations, n_components=3, obs_type="gaussian", n_iter=100):
    """
    Train a Hidden Semi-Markov Model (HSMM) on the provided observations.

    Parameters:
    - observations: List of sequences of observations (each sequence should be a 2D numpy array).
    - n_components: The number of states in the HSMM.
    - obs_type: Type of observation model ("gaussian", "bernoulli", etc.).
    - n_iter: Number of EM iterations for training.

    Returns:
    - model: Trained HSMM model.
    - mean_likelihood: Mean log-likelihood of the observation sequences.
    - std_likelihood: Standard deviation of log-likelihoods.
    - filtered_states: List of most likely states for each observation sequence.
    """
    # Flatten the data and create lengths array
    lengths = [len(seq) for seq in observations]
    X = np.concatenate(observations, axis=0)

    # Initialize HSMM
    hsmm = ssm.HSMM(K=n_components, D=X.shape[1], observations=obs_type,transition_kwargs={'r_min' : 1, 'r_max' : 3})

    # Fit the HSMM using the EM algorithm
    hsmm.fit(X, method="em")

    # Calculate log-likelihoods for each sequence
    log_likelihoods = [hsmm.log_likelihood(seq) for seq in observations]
    mean_likelihood = np.mean(log_likelihoods)
    std_likelihood = np.std(log_likelihoods)

    # Get filtered states for each sequence
    filtered_states = [hsmm.most_likely_states(seq) for seq in observations]

    return hsmm, mean_likelihood, std_likelihood, filtered_states


# Train HMMs for o and o1 observations
model_o, mean_likelihood_o, std_likelihood_o, filtered_states_o = train_hmm_and_evaluate(observations_o)
model_o1, mean_likelihood_o1, std_likelihood_o1, filtered_states_o1 = train_hmm_and_evaluate(observations_o1)

# Calculate state filtration errors
def calculate_filtration_error(filtered_states, true_states):
    total_errors = 0
    total_states = 0
    for filtered, true in zip(filtered_states, true_states):
        # print(((filtered == true)).shape)
        total_errors +=  (np.diag(filtered == true)).sum()
        total_states += len(true)
        # print(total_states)
    return total_errors / total_states

error_o = 0
error_o1 = 0

# Print results
print(f"HMM for 'o': Mean Likelihood = {mean_likelihood_o}, Std = {std_likelihood_o}, Filtration Error = {error_o}")
print(f"HMM for 'o1': Mean Likelihood = {mean_likelihood_o1}, Std = {std_likelihood_o1}, Filtration Error = {error_o1}")
# print(model_o.means_)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10408\3759679442.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pred1 = torch.load('TRAIN_DATA_NON_LINEAR.pth')
C:\P

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

HMM for 'o': Mean Likelihood = -262.7108983834663, Std = 202.194702440002, Filtration Error = 0
HMM for 'o1': Mean Likelihood = -445.48971296350123, Std = 152.78704865320393, Filtration Error = 0


In [12]:
def rearrange_states(filtered_states, hmm_means):
    """
    Rearranges states based on the order of HMM means.
    
    Args:
        filtered_states (list of np.ndarray): List of filtered states.
        hmm_means (np.ndarray): The means of the Gaussian HMM components.
        
    Returns:
        list of np.ndarray: Rearranged filtered states.
    """
    # Get the new state order by sorting the means
    sorted_indices = np.argsort(hmm_means.flatten())
    state_mapping = {old: new for new, old in enumerate(sorted_indices)}

    # Rearrange states in the filtered results
    rearranged_states = []
    for states in filtered_states:
        rearranged_states.append(np.vectorize(state_mapping.get)(states))
    
    return rearranged_states

In [13]:
# Repeat training multiple times and collect metrics
import time
n_repeats = 10
results_o = []
results_o1 = []
time_o = []
time_o1 = []

for _ in range(n_repeats):
    start_time = time.time()
    model_o, mean_likelihood_o, std_likelihood_o, filtered_states_o = train_hmm_and_evaluate(observations_o)
    hmm_means = model_o.observations.mus
    filtered_states_o = rearrange_states(filtered_states_o, hmm_means)
    print(filtered_states_o[0].shape)
 
    elapsed_time = time.time() - start_time
    error_o = 0
    results_o.append((mean_likelihood_o, std_likelihood_o, error_o))
    time_o.append(elapsed_time)

    start_time = time.time()
    model_o1, mean_likelihood_o1, std_likelihood_o1, filtered_states_o1 = train_hmm_and_evaluate(observations_o1)
    hmm_means = model_o1.observations.mus
    filtered_states_o1 = rearrange_states(filtered_states_o1, hmm_means)
    elapsed_time = time.time() - start_time
    error_o1 =0
    results_o1.append((mean_likelihood_o1, std_likelihood_o1, error_o1))
    time_o1.append(elapsed_time)

# Calculate mean and standard deviation for metrics
metrics_o = np.array(results_o)
metrics_o1 = np.array(results_o1)

mean_metrics_o = metrics_o.mean(axis=0)
std_metrics_o = metrics_o.std(axis=0)

mean_metrics_o1 = metrics_o1.mean(axis=0)
std_metrics_o1 = metrics_o1.std(axis=0)

# Calculate mean and standard deviation for computation time
mean_time_o = np.mean(time_o)
std_time_o = np.std(time_o)

mean_time_o1 = np.mean(time_o1)
std_time_o1 = np.std(time_o1)

# Print results
print("HMM for 'o':")
print(f"Mean Likelihood: {mean_metrics_o[0]}, Std: {std_metrics_o[0]}")
print(f"Mean Filtration Error: {mean_metrics_o[2]}, Std: {std_metrics_o[2]}")
print(f"Mean Computation Time: {mean_time_o}s, Std: {std_time_o}s\n")

print("HMM for 'o1':")
print(f"Mean Likelihood: {mean_metrics_o1[0]}, Std: {std_metrics_o1[0]}")
print(f"Mean Filtration Error: {mean_metrics_o1[2]}, Std: {std_metrics_o1[2]}")
print(f"Mean Computation Time: {mean_time_o1}s, Std: {std_time_o1}s")

  0%|          | 0/100 [00:00<?, ?it/s]

(258,)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

(258,)


...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

(258,)


  0%|          | 0/100 [00:00<?, ?it/s]

HMM for 'o':
Mean Likelihood: -262.8365253099457, Std: 0.16609351313759796
Mean Filtration Error: 0.0, Std: 0.0
Mean Computation Time: 9.681898355484009s, Std: 1.1569242735513425s

HMM for 'o1':
Mean Likelihood: -445.5044850060731, Std: 0.014158879269948261
Mean Filtration Error: 0.0, Std: 0.0
Mean Computation Time: 17.18494584560394s, Std: 1.7851928078866481s


In [14]:
# Print results
print("HMM for 'TOTAL':")
print(f"Mean Likelihood: {(metrics_o1+metrics_o).mean(axis=0)[0]}, Std: {(metrics_o1+metrics_o).std(axis=0)[0]}")

HMM for 'TOTAL':
Mean Likelihood: -708.3410103160187, Std: 0.1587919454230567


In [15]:
results_o

[(-262.7293031680149, 202.53578301569422, 0),
 (-262.7062874449234, 202.46274596949726, 0),
 (-263.07957689567274, 202.27930266154837, 0),
 (-263.0860168471501, 202.31785279881947, 0),
 (-263.1008875700785, 202.16166859734673, 0),
 (-262.7414701456202, 202.53463676496665, 0),
 (-262.7357203317769, 202.5284204863636, 0),
 (-262.7621273200009, 202.5643705259382, 0),
 (-262.72651295715093, 202.4346251343112, 0),
 (-262.6973504190684, 202.50375763495194, 0)]

In [16]:
results_o1

[(-445.52129464073494, 152.77706544589157, 0),
 (-445.5033308132479, 152.79916522276002, 0),
 (-445.47868047826677, 152.77088278731736, 0),
 (-445.48504097507924, 152.78898386112658, 0),
 (-445.5132368225338, 152.77844625392453, 0),
 (-445.5101698349952, 152.80818675140733, 0),
 (-445.50021292706384, 152.77954601228024, 0),
 (-445.5152267175706, 152.77260000005955, 0),
 (-445.4947461412921, 152.78831790928155, 0),
 (-445.52291070994715, 152.77811288375483, 0)]

# SINGLE MODEL 

In [17]:
observations_o = [np.concatenate([o.numpy().reshape(-1, 1),o1.numpy().reshape(-1, 1)], axis=1) for o, o1, _,_ in pred1]
# observations_o = [np.concatenate([o.numpy().reshape(-1, 1),o1.numpy().reshape(-1, 1)], axis=1) for o, o1, _,_ in pred1]

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
# Repeat training multiple times and collect metrics
import time
n_repeats = 10
results_o = []
results_o1 = []
time_o = []
time_o1 = []

for _ in range(n_repeats):
    start_time = time.time()
    model_o, mean_likelihood_o, std_likelihood_o, filtered_states_o = train_hmm_and_evaluate(observations_o, n_components=4)
    hmm_means = model_o.observations.mus
    filtered_states_o = rearrange_states(filtered_states_o, hmm_means)
    elapsed_time = time.time() - start_time
    error_o = 0
    results_o.append((mean_likelihood_o, std_likelihood_o, error_o))
    time_o.append(elapsed_time)

    start_time = time.time()

# Calculate mean and standard deviation for metrics
metrics_o = np.array(results_o)


mean_metrics_o = metrics_o.mean(axis=0)
std_metrics_o = metrics_o.std(axis=0)


# Calculate mean and standard deviation for computation time
mean_time_o = np.mean(time_o)
std_time_o = np.std(time_o)


# Print results
print("HMM for 'o':")
print(f"Mean Likelihood: {mean_metrics_o[0]}, Std: {std_metrics_o[0]}")
print(f"Mean Filtration Error: {mean_metrics_o[2]}, Std: {std_metrics_o[2]}")
print(f"Mean Computation Time: {mean_time_o}s, Std: {std_time_o}s\n")


In [ ]:
# Repeat training multiple times and collect metrics
import time
n_repeats = 10
results_o = []
results_o1 = []
time_o = []
time_o1 = []

for _ in range(n_repeats):
    start_time = time.time()
    model_o, mean_likelihood_o, std_likelihood_o, filtered_states_o = train_hmm_and_evaluate(observations_o, n_components=8)
    hmm_means = model_o.observations.mus
    filtered_states_o = rearrange_states(filtered_states_o, hmm_means)
    elapsed_time = time.time() - start_time
    error_o = 0
    results_o.append((mean_likelihood_o, std_likelihood_o, error_o))
    time_o.append(elapsed_time)

    start_time = time.time()

# Calculate mean and standard deviation for metrics
metrics_o = np.array(results_o)


mean_metrics_o = metrics_o.mean(axis=0)
std_metrics_o = metrics_o.std(axis=0)


# Calculate mean and standard deviation for computation time
mean_time_o = np.mean(time_o)
std_time_o = np.std(time_o)


# Print results
print("HMM for 'o':")
print(f"Mean Likelihood: {mean_metrics_o[0]}, Std: {std_metrics_o[0]}")
print(f"Mean Filtration Error: {mean_metrics_o[2]}, Std: {std_metrics_o[2]}")
print(f"Mean Computation Time: {mean_time_o}s, Std: {std_time_o}s\n")


In [ ]:
# Repeat training multiple times and collect metrics
import time
n_repeats = 10
results_o = []
results_o1 = []
time_o = []
time_o1 = []

for _ in range(n_repeats):
    start_time = time.time()
    model_o, mean_likelihood_o, std_likelihood_o, filtered_states_o = train_hmm_and_evaluate(observations_o, n_components=16)
    hmm_means = model_o.observations.mus
    filtered_states_o = rearrange_states(filtered_states_o, hmm_means)
    elapsed_time = time.time() - start_time
    error_o = 0
    results_o.append((mean_likelihood_o, std_likelihood_o, error_o))
    time_o.append(elapsed_time)

    start_time = time.time()

# Calculate mean and standard deviation for metrics
metrics_o = np.array(results_o)


mean_metrics_o = metrics_o.mean(axis=0)
std_metrics_o = metrics_o.std(axis=0)


# Calculate mean and standard deviation for computation time
mean_time_o = np.mean(time_o)
std_time_o = np.std(time_o)


# Print results
print("HMM for 'o':")
print(f"Mean Likelihood: {mean_metrics_o[0]}, Std: {std_metrics_o[0]}")
print(f"Mean Filtration Error: {mean_metrics_o[2]}, Std: {std_metrics_o[2]}")
print(f"Mean Computation Time: {mean_time_o}s, Std: {std_time_o}s\n")
